In [1]:
%%time

import requests
from bs4 import BeautifulSoup
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re


start_year = 2005
end_year = 2021

Years = range(start_year,end_year+1)

Countries = ['Portugal','Spain','England','Italy','Germany','France']

CPU times: user 688 ms, sys: 270 ms, total: 958 ms
Wall time: 1.14 s


In [204]:
%%time

# creation of an empty data frame for all countries, all years
table_fifa_all_years = pd.DataFrame()

for Year in Years:

    year = str(Year)[-2:]

    # creation of an empty data frame for all countries, one year
    table_fifa_all_countries = pd.DataFrame()

    for Country in Countries:
        if Country == 'Portugal':
            league = 308
        elif Country == 'Spain':
            league = 53
        elif Country == 'England':
            league = 13
        elif Country == 'Italy':
            league = 31
        elif Country == 'Germany':
            league = 19
        elif Country == 'France':
            league = 16
        else:
            print('Country not available, please choose between Portugal,\
        Spain, England, Italy, Germany or France')



        # Per each country

        URL = f'https://www.fifaindex.com/teams/fifa{year}/?league={league}&order=desc'

        # Ratings
        webpage = pd.read_html(URL)
        table_fifa_country = pd.DataFrame(webpage[0])
        table_fifa_country.dropna(subset = ["Name"], inplace=True)
        table_fifa_country.drop(table_fifa_country.columns.difference(
            ['Name','ATT','MID','DEF','OVR']), 1, inplace=True)
        table_fifa_country.insert(0, "Year", Year, True)
        table_fifa_country.insert(1, "Country", Country, True)
        table_fifa_country.reset_index(drop=True,inplace=True)

        # Rivals Teams & Budget
        r = requests.get(URL)
        page = r.content
        soup = BeautifulSoup(page, 'html5lib')

        teams_URL_code_in_national_league = []

        for a in soup.find_all('a', class_ = "link-team"):
            if a['href'] not in teams_URL_code_in_national_league:
                teams_URL_code_in_national_league.append(a['href'])

        table_fifa_country["Link-team"] = teams_URL_code_in_national_league


        Rival_team_list = []
        Team_budget_list = []
        Team_budget_list_aux = []

        for link_team in list(table_fifa_country["Link-team"]):
            URL = f'https://www.fifaindex.com{link_team}'
            r = requests.get(URL)
            page = r.content
            soup = BeautifulSoup(page, 'html5lib')
            li_class = soup.find_all('li',class_="list-group-item")


            for li_class_element in li_class:
                try:
                    rival_team = li_class_element.find("a", class_="link-team")
                    Rival_team_list.append(rival_team.text)
                except:
                    pass




# Fifa index does not show the budget for any team for the 2005 and 2006 years,
# so we will show the same budget for these years as the budget of 2007

            if ((Year == 2005) or (Year == 2006)):

                URL_original = f'https://www.fifaindex.com{link_team}'

                URL = URL_original[:-3] + '07/'

                r = requests.get(URL)
                page = r.content
                soup = BeautifulSoup(page, 'html5lib')
                li_class = soup.find_all('li',class_="list-group-item")

                for li_class_element in li_class:
                    try:
                        Budget_original = li_class_element.find("span", class_="data-currency data-currency-euro float-right").text
                        Budget_digits = re.findall(r'\d',Budget_original)
                        Budget = int(''.join(Budget_digits))/1000000

                        Team_budget_list_aux.append(Budget)

                    except:
                        pass

                if len(Team_budget_list_aux) == 0:
                    Team_budget_list_aux = [0]
                    Team_budget_list = Team_budget_list + Team_budget_list_aux
                    Team_budget_list_aux = []
                else:
                    Team_budget_list = Team_budget_list + Team_budget_list_aux
                    Team_budget_list_aux = []


            else:    


                for li_class_element in li_class:
                    try:
                        Budget_original = li_class_element.find("span", class_="data-currency data-currency-euro float-right").text
                        Budget_digits = re.findall(r'\d',Budget_original)
                        Budget = int(''.join(Budget_digits))/1000000

                        Team_budget_list_aux.append(Budget)

                    except:
                        pass

                if len(Team_budget_list_aux) == 0:
                    Team_budget_list_aux = [0]
                    Team_budget_list = Team_budget_list + Team_budget_list_aux
                    Team_budget_list_aux = []
                else:
                    Team_budget_list = Team_budget_list + Team_budget_list_aux
                    Team_budget_list_aux = []


        table_fifa_country["Rival_team"] = Rival_team_list
        table_fifa_country["Budget_Mill_€"] = Team_budget_list


        # Concat the table created for one team with the empty dataframe
        table_fifa_all_countries = pd.concat(
                [table_fifa_all_countries,
                 table_fifa_country],
            ignore_index = True, axis = 0)


    # Concat the table created for one year with the empty dataframe
    table_fifa_all_years = pd.concat(
            [table_fifa_all_years,
             table_fifa_all_countries],
         ignore_index = True, axis = 0)



table_fifa_all_years.to_excel(
    'Table_fifa_2005_2021.xlsx')    

table_fifa_all_years

CPU times: user 5min 27s, sys: 3.13 s, total: 5min 30s
Wall time: 10min 3s


,Year,Country,Name,ATT,MID,DEF,OVR,Link-team,Rival_team,Budget_Mill_€
0,2005,Portugal,FC Porto,85.0,82.0,77.0,81.0,/team/236/fc-porto/fifa05/,Benfica,16.0
1,2005,Portugal,Benfica,80.0,74.0,74.0,75.0,/team/234/benfica/fifa05/,Sporting Lisbon,5.0
2,2005,Portugal,Sporting Lisbon,63.0,70.0,65.0,68.0,/team/237/sporting-lisbon/fifa05/,Benfica,1.7
3,2005,Portugal,Belenenses,67.0,62.0,62.0,64.0,/team/1889/belenenses/fifa05/,Boavista,0.4
4,2005,Portugal,Nacional,69.0,59.0,61.0,63.0,/team/1891/nacional/fifa05/,Marítimo,0.5
...,...,...,...,...,...,...,...,...,...,...
1947,2021,France,FC Nantes,71.0,74.0,74.0,73.0,/team/71/fc-nantes/fifa21/,Stade Rennais FC,8.5
1948,2021,France,Nîmes Olympique,71.0,73.0,73.0,73.0,/team/224/n%C3%AEmes-olympique/fifa21/,Montpellier Hérault SC,4.8
1949,2021,France,FC Metz,70.0,70.0,73.0,72.0,/team/68/fc-metz/fifa21/,AS Nancy Lorraine,6.0
1950,2021,France,Dijon FCO,73.0,71.0,70.0,71.0,/team/110569/dijon-fco/fifa21/,AJ Auxerre,4.6


In [207]:
%%time

# Check if the number of teams per year and league of Fifa Index site
# matches the number of teams in the original table

table_all_years = pd.read_excel('All_teams_results_leagues_countries_2005_2021_edited.xlsx')
table_all_years = table_all_years.drop(['Unnamed: 0'], axis = 1)

info_all_missing_teams = []

for Year in Years:
    for Country in Countries:
        df_teams_fifa = table_fifa_all_years[((
        table_fifa_all_years['Year'] == int(f'{Year}')) &
        (table_fifa_all_years['Country'] == f'{Country}'
    ))]
        list_teams_fifa = list(set(list(df_teams_fifa['Name'])))
        len(list_teams_fifa)

        df_teams = table_all_years[(
        (table_all_years['Year'] == int(f'{Year}')) &
        (table_all_years['Country'] == f'{Country}') &
        (table_all_years['Competition'] == 'National League') 
    )]
        list_teams = list(set(list(df_teams['Home_team'])+list(df_teams['Away_team'])))
        len(list_teams)
        
        if (len(list_teams_fifa)-len(list_teams)) < 0:
            info_missing_teams = (Year,Country,f'Number of teams missing in FIFA table: {-(len(list_teams_fifa)-len(list_teams))}')
            info_all_missing_teams.append(list(info_missing_teams))
        elif (len(list_teams_fifa)-len(list_teams)) > 0:
            info_missing_teams = (Year,Country,f'Number of teams missing in original table: {len(list_teams_fifa)-len(list_teams)}')
            info_all_missing_teams.append(list(info_missing_teams))



# Get the lists of teams from original and fifa tables, for each year and country
# with missing teams in the fifa table

for teams_missing in info_all_missing_teams:
    
    year_teams_missing = teams_missing[0]
    country_teams_missing = teams_missing[1]
    
    aux_table = table_all_years[(table_all_years['Year']==int(year_teams_missing)) & 
                                   (table_all_years['Country']==country_teams_missing) & 
                                   (table_all_years['Competition']=='National League')
                               ]
    original_list = list(set(list(aux_table['Home_team'])+list(aux_table['Away_team'])))
    

    
    
    aux_table = table_fifa_all_years[(table_fifa_all_years['Year']==int(year_teams_missing)) & 
                                   (table_fifa_all_years['Country']==country_teams_missing)] 
                               
    fifa_list = list(set(aux_table['Name']))
    

    



# Create an auxiliary dataframe (df1) to find a match between an Origianl table
# team name, and a fifa table name, applying the library fuzzywuzzy with a high
# level of fuzzywuzzy score

    d = {'Name_from_original': original_list}
    df1 = pd.DataFrame(data=d)

    f = {'Name_from_fifa': fifa_list}
    df2 = pd.DataFrame(data=f)


    
    
    df1['Name_from_fifa_suggested_by_fuzzywuzzy'] = df1['Name_from_original'].apply(
        (lambda x: process.extractOne(x, df2['Name_from_fifa'].to_list())[0])
    )
    
    df1['fuzzywuzzy_score'] = df1['Name_from_original'].apply(
        (lambda x: process.extractOne(x, df2['Name_from_fifa'].to_list())[1])
    )
    
    fuzzywuzzy_min_score = 90
    
    df1['Suggested_name_with_confidence'] = df1.apply(
        lambda x: x['Name_from_fifa_suggested_by_fuzzywuzzy'] 
        if x['fuzzywuzzy_score'] >= fuzzywuzzy_min_score
        else 'NONE', axis = 1
    )


# Identify what teams do not have a high wuzzyfuzzy score

    list_original_teams_not_paired = []
    list_fifa_teams_not_paired = []
    
    for original_team in original_list:
        if original_team in list(df1[df1['Suggested_name_with_confidence']=='NONE']['Name_from_original']):
            list_original_teams_not_paired.append(original_team)
                                     
    for fifa_team in fifa_list:
        if fifa_team not in list(df1['Suggested_name_with_confidence']):
            list_fifa_teams_not_paired.append(fifa_team)

# Remove from the list the fifa teams that had already been assigned to an
# original team

    for fifa_team in list_fifa_teams_not_paired:
        if fifa_team in list(df1['Suggested_name_with_confidence']):
            list_fifa_teams_not_paired.remove(fifa_team)
                             

    
    
    
    

# Find the best match between the teams of fifa that had not been assigned yet
# to an original table team
    
    df1['Final_suggested_name'] = df1['Suggested_name_with_confidence']
    
    for fifa_team in list_fifa_teams_not_paired:
        original_name = process.extractOne(fifa_team, list_original_teams_not_paired)[0]
        index = df1[df1['Name_from_original']==original_name].index[0]
        df1.at[index,'Final_suggested_name'] = fifa_team


# Identify what original table teams name had been left alone (with no corresponding
# fifa team)

    original_names_finally_missing = []
    
    for i in range(len(list(df1['Final_suggested_name']))):
        if df1['Final_suggested_name'][i] == 'NONE':
            original_names_finally_missing.append(df1['Name_from_original'][i])
    

    
    
# Get the fifa name and fifa link team that corresponds to an original table team
# left alone
    
    fifa_teams_list_year_country = list(set(
        list(table_fifa_all_years[
                                  table_fifa_all_years['Country']==country_teams_missing
                                 ]['Name']) +\
    list(table_fifa_all_years[ 
                                  table_fifa_all_years['Country']==country_teams_missing
                                 ]['Rival_team'])))
    
    
    for original_team in original_names_finally_missing:
        fifa_name = process.extractOne(original_team, fifa_teams_list_year_country)[0]

    
        Link_team = table_fifa_all_years.loc[table_fifa_all_years['Name'] == fifa_name,'Link-team'].iloc[0]
        Link_team = Link_team[:-3] + str(year_teams_missing)[-2:] + '/'


# Now that we jave the Fifa_link, we need to get the rest of the info relative
# to the team, so we can add it to the fifa table

        URL = f'https://www.fifaindex.com{Link_team}'

        r = requests.get(URL)
        page = r.content
        soup = BeautifulSoup(page, 'html5lib')
        li_class = soup.find_all('li',class_="list-group-item")
        
        
        # Ratings
        
        for li_class_element in li_class:
            if li_class_element.text.startswith("Attack"):
                ATT = int(li_class_element.text[-2:])
            elif li_class_element.text.startswith("Midfield"):
                MID = int(li_class_element.text[-2:])
            elif li_class_element.text.startswith("Defence"):
                DEF = int(li_class_element.text[-2:])

        
        # We don't have the overall rating, so we can assume it as the average
        # of the 3 other ratings
        
        print(original_team)
        print(ATT,MID,DEF)
            
        OVR = round(((ATT + MID + DEF)+0.0001)/3)
            
        
        # Rivals Teams

        for li_class_element in li_class:
            try:
                rival_team = li_class_element.find("a", class_="link-team")
                Rival_team = rival_team.text
            except:
                pass


        
        
        # Budget
        
        if ((year_teams_missing == 2005) or (year_teams_missing == 2006)):

            URL = URL_original[:-3] + '07/'

            r = requests.get(URL)
            page = r.content
            soup = BeautifulSoup(page, 'html5lib')
            li_class = soup.find_all('li',class_="list-group-item")

            for li_class_element in li_class:
                try:
                    Budget_original = li_class_element.find("span", class_="data-currency data-currency-euro float-right").text
                    Budget_digits = re.findall(r'\d',Budget_original)
                    Budget = int(''.join(Budget_digits))/1000000

                except:
                    pass
                    
        else:    
            for li_class_element in li_class:
                try:
                    Budget_original = li_class_element.find("span", class_="data-currency data-currency-euro float-right").text
                    Budget_digits = re.findall(r'\d',Budget_original)
                    Budget = int(''.join(Budget_digits))/1000000
                except:
                    pass



# Add the missing information collected to the fifa table

        new_row = {'Year':year_teams_missing,
                   'Country':country_teams_missing,
                   'Name': fifa_name,
                   'ATT':ATT,
                   'MID':MID,
                   'DEF':DEF,
                   'OVR':OVR,
                   'Link-team':Link_team,
                   'Rival_team':Rival_team,
                   'Budget_Mill_€':Budget
                  }

        table_fifa_all_years = table_fifa_all_years.append(new_row, ignore_index=True)




        
table_fifa_all_years

CPU times: user 15.2 s, sys: 93.4 ms, total: 15.3 s
Wall time: 15.5 s


,Year,Country,Name,ATT,MID,DEF,OVR,Link-team,Rival_team,Budget_Mill_€,Team-ID
0,2005,Portugal,FC Porto,85,82,77,81,/team/236/fc-porto/fifa05/,Benfica,16.0,236
1,2005,Portugal,Benfica,80,74,74,75,/team/234/benfica/fifa05/,Sporting Lisbon,5.0,234
2,2005,Portugal,Sporting Lisbon,63,70,65,68,/team/237/sporting-lisbon/fifa05/,Benfica,1.7,237
3,2005,Portugal,Belenenses,67,62,62,64,/team/1889/belenenses/fifa05/,Boavista,0.4,1889
4,2005,Portugal,Nacional,69,59,61,63,/team/1891/nacional/fifa05/,Marítimo,0.5,1891
...,...,...,...,...,...,...,...,...,...,...,...
1951,2021,France,FC Lorient,71,71,70,71,/team/217/fc-lorient/fifa21/,Stade Rennais FC,5.0,217
1952,2012,England,Arsenal,84,80,81,82,/team/1/arsenal/fifa12/,Tottenham Hotspur,30.0,1
1953,2013,England,Southampton,73,74,71,73,/team/17/southampton/fifa13/,Portsmouth,7.5,17
1954,2014,Spain,Osasuna,71,71,73,72,/team/479/osasuna/fifa14/,Athletic Club de Bilbao,4.8,479


In [209]:
table_fifa_all_years = pd.read_excel('Table_fifa_2005_2021.xlsx')
table_fifa_all_years = table_fifa_all_years.drop(['Unnamed: 0'], axis = 1)

table_fifa_all_years['Team-ID'] = table_fifa_all_years['Link-team'].apply(
lambda x: x.split("/")[2]
)

list_all_names = []

for ID in list(table_fifa_all_years['Team-ID']):
    list_all_names.append(list(pd.unique(table_fifa_all_years[table_fifa_all_years['Team-ID']==ID]['Name'])))

table_fifa_all_years['Fifa_team_all_names'] = list_all_names


table_fifa_all_years

,Year,Country,Name,ATT,MID,DEF,OVR,Link-team,Rival_team,Budget_Mill_€,Team-ID,Fifa_team_all_names
0,2005,Portugal,FC Porto,85,82,77,81,/team/236/fc-porto/fifa05/,Benfica,16.0,236,"[FC Porto, F.C. Porto]"
1,2005,Portugal,Benfica,80,74,74,75,/team/234/benfica/fifa05/,Sporting Lisbon,5.0,234,"[Benfica, Sport Lisboa Benfica, SL Benfica]"
2,2005,Portugal,Sporting Lisbon,63,70,65,68,/team/237/sporting-lisbon/fifa05/,Benfica,1.7,237,"[Sporting Lisbon, Sporting CP Lisbon, Sporting..."
3,2005,Portugal,Belenenses,67,62,62,64,/team/1889/belenenses/fifa05/,Boavista,0.4,1889,"[Belenenses, CF Os Belenenses, Belém, C.F. Os ..."
4,2005,Portugal,Nacional,69,59,61,63,/team/1891/nacional/fifa05/,Marítimo,0.5,1891,"[Nacional, Clube Desportivo Nacional, CD Nacio..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1951,2021,France,FC Lorient,71,71,70,71,/team/217/fc-lorient/fifa21/,Stade Rennais FC,5.0,217,"[Lorient, FC Lorient Bretagne Sud, FC Lorient]"
1952,2012,England,Arsenal,84,80,81,82,/team/1/arsenal/fifa12/,Tottenham Hotspur,30.0,1,"[Arsenal, Arsenal FC]"
1953,2013,England,Southampton,73,74,71,73,/team/17/southampton/fifa13/,Portsmouth,7.5,17,[Southampton]
1954,2014,Spain,Osasuna,71,71,73,72,/team/479/osasuna/fifa14/,Athletic Club de Bilbao,4.8,479,"[Osasuna, Club Atlético Osasuna, CA Osasuna]"


In [231]:
team = 'Inter'

list(table_fifa_all_years.loc[table_fifa_all_years['Name']==team]['Fifa_team_all_names'])[0]

['Inter Milan', 'Inter']

In [201]:
table_fifa_all_years['Fifa_team_all_names'][99]

['Paris Saint-Germain']

In [203]:
table_fifa_all_years = pd.read_excel('Table_fifa_2005_2021.xlsx')
table_fifa_all_years = table_fifa_all_years.drop(['Unnamed: 0'], axis = 1)
table_fifa_all_years

,Year,Country,Name,ATT,MID,DEF,OVR,Link-team,Rival_team,Budget_Mill_€,Team-ID,Fifa_Team_names,Fifa_team_all_names
0,2005,Portugal,FC Porto,85,82,77,81,/team/236/fc-porto/fifa05/,Benfica,16.0,236,['FC Porto'],"['FC Porto', 'F.C. Porto']"
1,2005,Portugal,Benfica,80,74,74,75,/team/234/benfica/fifa05/,Sporting Lisbon,5.0,234,['Benfica'],"['Benfica', 'Sport Lisboa Benfica', 'SL Benfica']"
2,2005,Portugal,Sporting Lisbon,63,70,65,68,/team/237/sporting-lisbon/fifa05/,Benfica,1.7,237,['Sporting Lisbon'],"['Sporting Lisbon', 'Sporting CP Lisbon', 'Spo..."
3,2005,Portugal,Belenenses,67,62,62,64,/team/1889/belenenses/fifa05/,Boavista,0.4,1889,['Belenenses'],"['Belenenses', 'CF Os Belenenses', 'Belém', 'C..."
4,2005,Portugal,Nacional,69,59,61,63,/team/1891/nacional/fifa05/,Marítimo,0.5,1891,['Nacional'],"['Nacional', 'Clube Desportivo Nacional', 'CD ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1951,2021,France,FC Lorient,71,71,70,71,/team/217/fc-lorient/fifa21/,Stade Rennais FC,5.0,217,['FC Lorient'],"['Lorient', 'FC Lorient Bretagne Sud', 'FC Lor..."
1952,2012,England,Arsenal,84,80,81,82,/team/1/arsenal/fifa12/,Tottenham Hotspur,30.0,1,['Arsenal'],"['Arsenal', 'Arsenal FC']"
1953,2013,England,Southampton,73,74,71,73,/team/17/southampton/fifa13/,Portsmouth,7.5,17,['Southampton'],['Southampton']
1954,2014,Spain,Osasuna,71,71,73,72,/team/479/osasuna/fifa14/,Athletic Club de Bilbao,4.8,479,['Osasuna'],"['Osasuna', 'Club Atlético Osasuna', 'CA Osasu..."


In [6]:
table_all_years = pd.read_excel('All_teams_results_leagues_countries_2005_2021_edited.xlsx')
table_all_years = table_all_years.drop(['Unnamed: 0'], axis = 1)
table_all_years.head(3)

,Year,Country,Date,Datetime_date_list,Competition_original_name_URL,Competition,Home_team,Away_team,Result,Home_score,Away_score,1x2,Points_Home_Team,Points_Away_Team,number_of_games_last_days_home_team,number_of_games_last_days_away_team
0,2005,International,24 Feb 05,2005-02-24,Europa League,Europa League,Benfica,CSKA Moskva,1 - 1,1,1,x,1,1,4,1
1,2005,International,17 Feb 05,2005-02-17,Europa League,Europa League,CSKA Moskva,Benfica,2 - 0,2,0,1,3,0,0,3
2,2005,International,02 Dec 04,2004-12-02,Europa League,Europa League,KSK Beveren,Benfica,0 - 3,0,3,2,0,3,0,4


In [155]:
%%time

import collections

d = collections.defaultdict(list)

# Build 2 lists: one with unique values of all teams in original table and
# another with unique values of all teams in fifa table

df1 = table_all_years[table_all_years['Competition']=='National League']

for year in Years:
    for country in Countries:
        original_teams_list_year_country = list(set(
            list(df1[(df1['Year'] == year) & 
                     (df1['Country'] == country)]
                     ['Home_team']) +\
            list(df1[(df1['Year'] == year) & 
                     (df1['Country'] == country)]
                     ['Away_team'])))

        fifa_teams_list_year_country = list(set(
            list(table_fifa_all_years[(table_fifa_all_years['Year'] == year) & 
                     (table_fifa_all_years['Country'] == country)]
                     ['Name'])))
        
        print(year)
        print(country)

        

# Build a dictionary collection with keys being the team names in the original
# table, and values the different names in the fifa table of each original team.
        

        
        if len(original_teams_list_year_country) != len(fifa_teams_list_year_country):
            print('********************************************************')
            print('********************************************************')
            print('********************************************************')
            print(f'ERROR: {year},{country},\n\
            Number of teams original table: {len(original_teams_list_year_country)},\n\
            Number of teams fifa table: {len(fifa_teams_list_year_country)}')
            print('********************************************************')
            print('********************************************************')
            print('********************************************************')
            print('\n')
        
        else:

            # Then we create 2 lists: one with the original teams that have already
            # been matched with a fifa team, and another with the ones that have not been
            # yet matched. Then ,we do the same with the fifa teams: we build a list with 
            # the ones that had already been assigned to an original team, and another with
            # the teams that have not been assigned yet.

            

            print('\n')
            print('original_teams_list_year_country:\n',original_teams_list_year_country)
            print('\n')
            print('fifa_teams_list_year_country:\n',fifa_teams_list_year_country)
            print('\n')
            
                
            data = {'Original_name':original_teams_list_year_country}
            df = pd.DataFrame(data)
            
            df['Fifa_name_suggested'] = df['Original_name'].apply(
        (lambda x: process.extractOne(x, fifa_teams_list_year_country)[0])
    )
            
            df['Fifa_name_suggested_fuzzywuzzy_score'] = df['Original_name'].apply(
        (lambda x: process.extractOne(x, fifa_teams_list_year_country)[1])
    )
            
            df.sort_values(by = ['Fifa_name_suggested_fuzzywuzzy_score'], ascending = False,
                          inplace = True)
            
            original_teams_list_year_country = list(df['Original_name'])
            
  

            # Here we look for the best match between an original team name and a fifa team
            # name. For that we run different loops with different levels of fuzzywuzzy
            # score.
            
            #fifa_teams_list_year_country_not_in_d = []
                    
            #for fifa_team in fifa_teams_list_year_country:
            #    if fifa_team not in sum(d.values(),[]):
            #        fifa_teams_list_year_country_not_in_d.append(fifa_team)
            
            #print(fifa_teams_list_year_country_not_in_d)
            
            
            
            fuzzywuzzy_score_level = [100,90,80,70,60,50,40,30,20,10,0]
            
            for original_team in original_teams_list_year_country:
                
                for score_level in fuzzywuzzy_score_level:
                
                    fuzzywuzzy_min_score = score_level

                    fifa_team_suggested = process.extractOne(original_team, 
                            fifa_teams_list_year_country)[0]
                    fifa_team_suggested_score = process.extractOne(original_team, 
                            fifa_teams_list_year_country)[1]


                    # Build a dictionary collection with keys being the team names in the original
                    # table, and values the different names in the fifa table of each original team.
                    

                    
                    if fifa_team_suggested_score >= fuzzywuzzy_min_score:
                        fifa_teams_list_year_country.remove(fifa_team_suggested)
                        
                        if fifa_team_suggested not in sum(d.values(),[]):
                            d[original_team].append(fifa_team_suggested)
                            d[original_team].append(year)
                        
                        
                        print(original_team,'-->',fifa_team_suggested,'-->',fifa_team_suggested_score)
                        
                        break

            print('-----------------------------')

2005
Portugal


original_teams_list_year_country:
 ['Os Belenenses', 'Rio Ave', 'Estoril', 'Boavista', 'Nacional', 'Gil Vicente', 'Porto', 'Benfica', 'Académica', 'Penafiel', 'Vitória Setúbal', 'Moreirense', 'Marítimo', 'União de Leiria', 'Beira Mar SC', 'Vitória Guimarães', 'Sporting CP', 'Sporting Braga']


fifa_teams_list_year_country:
 ['Coimbra', 'Rio Ave', 'Nacional', 'União Leiria', 'Boavista', 'Estoril Praia', 'Gil Vicente', 'Benfica', 'FC Porto', 'Sporting Lisbon', 'Vitória Futebol Clube', 'Belenenses', 'Marítimo', 'Sporting Braga', 'SC Beira-Mar', 'Moreirense', 'Vitória SC', 'F.C. Penafiel']


Sporting Braga --> Sporting Braga --> 100
Moreirense --> Moreirense --> 100
Boavista --> Boavista --> 100
Nacional --> Nacional --> 100
Gil Vicente --> Gil Vicente --> 100
Benfica --> Benfica --> 100
Marítimo --> Marítimo --> 100
Rio Ave --> Rio Ave --> 100
Beira Mar SC --> SC Beira-Mar --> 95
União de Leiria --> União Leiria --> 95
Os Belenenses --> Belenenses --> 95
Porto --> FC Porto

Athletic --> Athletic de Bilbao --> 90
Málaga --> Málaga CF --> 90
Racing --> Racing de Santander --> 90
R. Sociedad --> Real Sociedad --> 85
-----------------------------
2006
England


original_teams_list_year_country:
 ['Portsmouth', 'West Bromwich Albion', 'Chelsea', 'Birmingham City', 'Liverpool', 'Fulham', 'Tottenham Hotspur', 'Everton', 'Bolton Wanderers', 'Man. Utd', 'Sunderland', 'Middlesbrough', 'West Ham', 'Charlton Athletic', 'Wigan Athletic', 'Newcastle', 'Man. City', 'Arsenal', 'Aston Villa', 'Blackburn Rovers']


fifa_teams_list_year_country:
 ['Portsmouth', 'Manchester United', 'Chelsea', 'Manchester City', 'Birmingham City', 'Liverpool', 'West Bromwich', 'Fulham', 'West Ham United', 'Tottenham Hotspur', 'Everton', 'Bolton Wanderers', 'Sunderland', 'Newcastle United', 'Middlesbrough', 'Charlton Athletic', 'Wigan Athletic', 'Arsenal', 'Aston Villa', 'Blackburn Rovers']


Portsmouth --> Portsmouth --> 100
Bolton Wanderers --> Bolton Wanderers --> 100
Aston Villa --> Aston

Tottenham Hotspur --> Tottenham Hotspur --> 100
Fulham --> Fulham --> 100
Liverpool --> Liverpool --> 100
Watford --> Watford --> 100
Reading --> Reading FC --> 95
Newcastle --> Newcastle United --> 90
West Ham --> West Ham United --> 90
Man. City --> Manchester City --> 86
Man. Utd --> Manchester United --> 56
-----------------------------
2007
Italy


original_teams_list_year_country:
 ['Inter', 'Reggina', 'Sampdoria', 'Udinese', 'Siena', 'Lazio', 'Torino', 'Messina', 'Roma', 'Fiorentina', 'Catania', 'Ascoli', 'Chievo', 'Atalanta', 'Milan', 'Cagliari', 'Empoli', 'Livorno', 'Palermo FC', 'Parma']


fifa_teams_list_year_country:
 ['Atalanta BC', 'Reggina Calcio', 'Sampdoria', 'Udinese', 'Siena', 'Lazio', 'Torino', 'Chievo Verona', 'Messina', 'Palermo', 'Fiorentina', 'Inter Milan', 'Catania', 'Ascoli', 'Cagliari', 'Empoli', 'Livorno', 'AC Milan', 'Parma', 'AS Roma']


Catania --> Catania --> 100
Messina --> Messina --> 100
Livorno --> Livorno --> 100
Empoli --> Empoli --> 100
Cagliari -

Napoli --> Napoli --> 100
Fiorentina --> Fiorentina --> 100
Juventus --> Juventus --> 100
Empoli --> Empoli --> 100
Cagliari --> Cagliari --> 100
Livorno --> Livorno --> 100
Catania --> Catania --> 100
Parma --> Parma --> 100
Torino --> Torino --> 100
Lazio --> Lazio --> 100
Siena --> Siena --> 100
Genoa --> Genoa --> 100
Udinese --> Udinese --> 100
Sampdoria --> Sampdoria --> 100
Atalanta --> Atalanta BC --> 95
Palermo FC --> Palermo --> 95
Roma --> AS Roma --> 90
Reggina --> Reggina Calcio --> 90
Milan --> Inter Milan --> 90
Inter --> AC Milan --> 36
-----------------------------
2008
Germany


original_teams_list_year_country:
 ['Arminia Bielefeld', 'Hannover 96', 'Wolfsburg', 'Eintracht Frankfurt', 'MSV Duisburg', 'Nürnberg', 'Hamburger SV', 'VfL Bochum', 'Schalke 04', 'Stuttgart', 'Werder Bremen', 'Hansa Rostock', 'Energie Cottbus', 'Karlsruher SC', 'Hertha BSC', 'Bayern München', 'B. Leverkusen', 'B. Dortmund']


fifa_teams_list_year_country:
 ['Hertha BSC Berlin', 'Arminia Biele

Inter --> AC Milan --> 36
Genoa --> F. Genova --> 72
-----------------------------
2009
Germany


original_teams_list_year_country:
 ['Arminia Bielefeld', 'Hannover 96', 'Wolfsburg', 'B. Dortmund', 'Eintracht Frankfurt', 'Hamburger SV', 'Hertha BSC', 'VfL Bochum', 'Hoffenheim', 'Stuttgart', 'Schalke 04', 'Energie Cottbus', 'B. Leverkusen', 'Karlsruher SC', 'Bayern München', 'Werder Bremen', 'Köln', 'B. Mönchengladbach']


fifa_teams_list_year_country:
 ['Arminia Bielefeld', 'FC Schalke 04', 'Hannover 96', 'VfB Stuttgart', 'VfL Wolfsburg', 'Hamburger SV', 'Eintracht Frankfurt', 'VfL Bochum', '1899 Hoffenheim', "Borussia M'gladbach", 'Bayer 04 Leverkusen', 'Borussia Dortmund', '1. FC Köln', 'Karlsruher SC', 'Hertha BSC', 'Energie Cottbus', 'Bayern München', 'Werder Bremen']


Arminia Bielefeld --> Arminia Bielefeld --> 100
Energie Cottbus --> Energie Cottbus --> 100
Werder Bremen --> Werder Bremen --> 100
Bayern München --> Bayern München --> 100
Eintracht Frankfurt --> Eintracht Frankfu

...

Las Palmas --> UD Las Palmas --> 95
Real Madrid --> Real Madrid CF --> 95
Granada --> Granada CF --> 95
Osasuna --> CA Osasuna --> 95
Villarreal --> Villarreal CF --> 95
Barcelona --> FC Barcelona --> 95
Sevilla --> Sevilla FC --> 95
Valencia --> Valencia CF --> 95
Málaga --> Málaga CF --> 90
Athletic --> Athletic Club de Bilbao --> 90
Espanyol --> RCD Espanyol --> 90
Real Betis --> Real Betis Balompié --> 90
Atlético --> Atlético Madrid --> 90
Eibar --> SD Eibar --> 90
Leganés --> CD Leganés --> 90
Alavés --> Deportivo Alavés --> 90
Celta --> RC Celta de Vigo --> 90
Deportivo --> RC Deportivo de La Coruña --> 90
Real Sporting --> Real Sporting de Gijón --> 90
R. Sociedad --> Real Sociedad --> 85
-----------------------------
2017
England


original_teams_list_year_country:
 ['West Bromwich Albion', 'Chelsea', 'Watford', 'Liverpool', 'Hull City', 'Tottenham Hotspur', 'Everton', 'Swansea City', 'AFC Bournemouth', 'Man. Utd', 'Sunderland', 'Middlesbrough', 'West Ham', 'Man. City', 'Cryst

Alavés --> Deportivo Alavés --> 90
Eibar --> SD Eibar --> 90
Celta --> RC Celta de Vigo --> 90
Deportivo --> RC Deportivo de La Coruña --> 90
Real Betis --> Real Betis Balompié --> 90
Getafe --> Getafe CF --> 90
Espanyol --> RCD Espanyol --> 90
Athletic --> Athletic Club de Bilbao --> 90
Málaga --> Málaga CF --> 90
Atlético --> Atlético Madrid --> 90
R. Sociedad --> Real Sociedad --> 85
-----------------------------
2018
England


original_teams_list_year_country:
 ['West Bromwich Albion', 'Chelsea', 'Brighton & Hove Albion', 'Watford', 'Liverpool', 'Tottenham Hotspur', 'Everton', 'Huddersfield Town', 'Swansea City', 'AFC Bournemouth', 'Man. Utd', 'West Ham', 'Newcastle', 'Man. City', 'Crystal Palace', 'Stoke City', 'Southampton', 'Burnley', 'Arsenal', 'Leicester']


fifa_teams_list_year_country:
 ['Manchester United', 'West Bromwich Albion', 'Chelsea', 'Brighton & Hove Albion', 'Watford', 'Manchester City', 'Liverpool', 'Leicester City', 'West Ham United', 'Tottenham Hotspur', 'Everto

2019
Italy


original_teams_list_year_country:
 ['Inter', 'Sampdoria', 'Frosinone', 'Udinese', 'Genoa', 'Lazio', 'Torino', 'SPAL', 'Roma', 'Bologna', 'Fiorentina', 'Napoli', 'Chievo', 'Atalanta', 'Milan', 'Empoli', 'Cagliari', 'Sassuolo', 'Juventus', 'Parma']


fifa_teams_list_year_country:
 ['Inter', 'Sampdoria', 'Frosinone', 'Udinese', 'Genoa', 'Lazio', 'Torino', 'Chievo Verona', 'SPAL', 'Roma', 'Bologna', 'Fiorentina', 'Napoli', 'Atalanta', 'Milan', 'Empoli', 'Cagliari', 'Sassuolo', 'Juventus', 'Parma']


Inter --> Inter --> 100
Sampdoria --> Sampdoria --> 100
Juventus --> Juventus --> 100
Sassuolo --> Sassuolo --> 100
Cagliari --> Cagliari --> 100
Empoli --> Empoli --> 100
Milan --> Milan --> 100
Atalanta --> Atalanta --> 100
Napoli --> Napoli --> 100
Fiorentina --> Fiorentina --> 100
Bologna --> Bologna --> 100
Roma --> Roma --> 100
SPAL --> SPAL --> 100
Torino --> Torino --> 100
Lazio --> Lazio --> 100
Genoa --> Genoa --> 100
Udinese --> Udinese --> 100
Frosinone --> Frosinone --

B. Mönchengladbach --> Borussia Mönchengladbach --> 88
B. Leverkusen --> Bayer 04 Leverkusen --> 86
SC Freiburg --> Sport-Club Freiburg --> 86
B. Dortmund --> Borussia Dortmund --> 86
-----------------------------
2020
France


original_teams_list_year_country:
 ['Olympique Marseille', 'PSG', 'Nantes', 'Metz', 'Dijon FCO', 'Lille', 'Angers SCO', 'Amiens SC', 'Nice', 'Montpellier', 'Strasbourg', 'Monaco', 'Stade de Reims', 'Girondins Bordeaux', 'Stade Brestois', 'Saint-Étienne', 'Toulouse', 'Nîmes', 'Olympique Lyonnais', 'Stade Rennais']


fifa_teams_list_year_country:
 ['Toulouse Football Club', 'Dijon FCO', 'Nîmes Olympique', 'Paris Saint-Germain', 'FC Nantes', 'Angers SCO', 'Amiens SC', 'Stade Brestois 29', 'Montpellier Hérault SC', 'Football Club de Metz', 'FC Girondins de Bordeaux', 'Stade de Reims', 'Stade Rennais FC', 'Olympique de Marseille', 'AS Monaco Football Club SA', 'RC Strasbourg Alsace', 'Olympique Lyonnais', 'LOSC Lille', 'OGC Nice', 'AS Saint-Étienne']


Stade de Reims

2021
France


original_teams_list_year_country:
 ['Olympique Marseille', 'Lens', 'Metz', 'PSG', 'Nantes', 'Dijon FCO', 'Lille', 'Angers SCO', 'Nice', 'Montpellier', 'Strasbourg', 'Monaco', 'Stade de Reims', 'Girondins Bordeaux', 'Stade Brestois', 'Saint-Étienne', 'Nîmes', 'Olympique Lyonnais', 'Lorient', 'Stade Rennais']


fifa_teams_list_year_country:
 ['Nîmes Olympique', 'Dijon FCO', 'Paris Saint-Germain', 'FC Nantes', 'Angers SCO', 'Stade Brestois 29', 'Montpellier Hérault SC', 'FC Metz', 'RC Lens', 'FC Girondins de Bordeaux', 'Stade de Reims', 'Stade Rennais FC', 'FC Lorient', 'Olympique de Marseille', 'RC Strasbourg Alsace', 'AS Monaco', 'Olympique Lyonnais', 'LOSC Lille', 'OGC Nice', 'AS Saint-Étienne']


Stade de Reims --> Stade de Reims --> 100
Olympique Lyonnais --> Olympique Lyonnais --> 100
Dijon FCO --> Dijon FCO --> 100
Angers SCO --> Angers SCO --> 100
Olympique Marseille --> Olympique de Marseille --> 95
Lorient --> FC Lorient --> 95
Saint-Étienne --> AS Saint-Étienne --

In [156]:
print(d)

defaultdict(<class 'list'>, {'Sporting Braga': ['Sporting Braga', 2005, 'SC Braga', 2010], 'Moreirense': ['Moreirense', 2005, 'Moreira de Cónegos', 2013, 'Moreirense FC', 2015], 'Boavista': ['Boavista', 2005, 'Boavista Futebol Clube', 2008, 'Boavista FC', 2015], 'Nacional': ['Nacional', 2005, 'Clube Desportivo Nacional', 2008, 'CD Nacional', 2010, 'Funchal', 2019], 'Gil Vicente': ['Gil Vicente', 2005, 'Gil Vicente FC', 2020], 'Benfica': ['Benfica', 2005, 'Sport Lisboa Benfica', 2008, 'SL Benfica', 2016], 'Marítimo': ['Marítimo', 2005, 'Marítimo da Madeira', 2008, 'CS Marítimo', 2010, 'C. Funchal', 2011, 'F. Santa Maria da Feira', 2012], 'Rio Ave': ['Rio Ave', 2005, 'Rio Ave FC', 2010], 'Beira Mar SC': ['SC Beira-Mar', 2005, 'SC Beira Mar', 2011], 'União de Leiria': ['União Leiria', 2005, 'União Desportivo de Leiria', 2008, 'União de Leiria, SAD', 2010], 'Os Belenenses': ['Belenenses', 2005, 'CF Os Belenenses', 2010, 'Belém', 2014, 'C.F. Os Belenenses', 2015, 'Os Belenenses', 2019], 'Po

In [266]:
%%time

import collections

d = collections.defaultdict(list)

# Build 2 lists: one with unique values of all teams in original table and
# another with unique values of all teams in fifa table

df1 = table_all_years[table_all_years['Competition']=='National League']

for year in Years:
    for country in Countries:
        original_teams_list_year_country = list(set(
            list(df1[(df1['Year'] == year) & 
                     (df1['Country'] == country)]
                     ['Home_team']) +\
            list(df1[(df1['Year'] == year) & 
                     (df1['Country'] == country)]
                     ['Away_team'])))

        fifa_teams_list_year_country = list(set(
            list(table_fifa_all_years[(table_fifa_all_years['Year'] == year) & 
                     (table_fifa_all_years['Country'] == country)]
                     ['Name'])))
        
        print(year)
        print(country)

        

# Build a dictionary collection with keys being the team names in the original
# table, and values the different names in the fifa table of each original team.
        

        
        if len(original_teams_list_year_country) != len(fifa_teams_list_year_country):
            print('********************************************************')
            print('********************************************************')
            print('********************************************************')
            print(f'ERROR: {year},{country},\n\
            Number of teams original table: {len(original_teams_list_year_country)},\n\
            Number of teams fifa table: {len(fifa_teams_list_year_country)}')
            print('********************************************************')
            print('********************************************************')
            print('********************************************************')
            print('\n')
        
        else:

            # Then we create 2 lists: one with the original teams that have already
            # been matched with a fifa team, and another with the ones that have not been
            # yet matched. Then ,we do the same with the fifa teams: we build a list with 
            # the ones that had already been assigned to an original team, and another with
            # the teams that have not been assigned yet.

            

            #print('\n')
            #print('original_teams_list_year_country:\n',original_teams_list_year_country)
            #print('\n')
            #print('fifa_teams_list_year_country:\n',fifa_teams_list_year_country)
            #print('\n')
            
                
            data = {'Original_name':original_teams_list_year_country}
            df = pd.DataFrame(data)
            
            df['Fifa_name_suggested'] = df['Original_name'].apply(
        (lambda x: process.extractOne(x, fifa_teams_list_year_country)[0])
    )
            
            df['Fifa_name_suggested_fuzzywuzzy_score'] = df['Original_name'].apply(
        (lambda x: process.extractOne(x, fifa_teams_list_year_country)[1])
    )
            
            df.sort_values(by = ['Fifa_name_suggested_fuzzywuzzy_score'], ascending = False,
                          inplace = True)
            
            original_teams_list_year_country = list(df['Original_name'])
            
  

            # Here we look for the best match between an original team name and a fifa team
            # name. For that we run different loops with different levels of fuzzywuzzy
            # score.
            
            #fifa_teams_list_year_country_not_in_d = []
                    
            #for fifa_team in fifa_teams_list_year_country:
            #    if fifa_team not in sum(d.values(),[]):
            #        fifa_teams_list_year_country_not_in_d.append(fifa_team)
            
            #print(fifa_teams_list_year_country_not_in_d)
            
            
            
            fuzzywuzzy_score_level = [100,90,80,70,60,50,40,30,20,10,0]
            
            for original_team in original_teams_list_year_country:
                
                if original_team in d.keys():
                    continue
                
                else:

                    for score_level in fuzzywuzzy_score_level:

                        fuzzywuzzy_min_score = score_level

                        fifa_team_suggested = process.extractOne(original_team, 
                                fifa_teams_list_year_country)[0]
                        fifa_team_suggested_score = process.extractOne(original_team, 
                                fifa_teams_list_year_country)[1]


                        # Build a dictionary collection with keys being the team names in the original
                        # table, and values the different names in the fifa table of each original team.



                        if fifa_team_suggested_score >= fuzzywuzzy_min_score:
                            fifa_teams_list_year_country.remove(fifa_team_suggested)

                            if fifa_team_suggested not in sum(sum(d.values(),[]),[]):
                                
                                #print('\n')
                                #print(d.values())
                                #print(sum(d.values(),[]))
                                print(sum(sum(d.values(),[]),[]))
                                #print('\n')
                                
                                d[original_team].append(
                                list(table_fifa_all_years.loc[table_fifa_all_years['Name']==fifa_team_suggested]
                                     ['Fifa_team_all_names'])[0]
                                )


                            print(original_team,'-->',fifa_team_suggested,'-->',fifa_team_suggested_score)

                            break

            print('-----------------------------')

2005
Portugal
[]
Sporting Braga --> Sporting Braga --> 100
['Sporting Braga', 'SC Braga']
Moreirense --> Moreirense --> 100
['Sporting Braga', 'SC Braga', 'Moreirense', 'Moreira de Cónegos', 'Moreirense FC']
Boavista --> Boavista --> 100
['Sporting Braga', 'SC Braga', 'Moreirense', 'Moreira de Cónegos', 'Moreirense FC', 'Boavista', 'Boavista Futebol Clube', 'Boavista FC']
Nacional --> Nacional --> 100
['Sporting Braga', 'SC Braga', 'Moreirense', 'Moreira de Cónegos', 'Moreirense FC', 'Boavista', 'Boavista Futebol Clube', 'Boavista FC', 'Nacional', 'Clube Desportivo Nacional', 'CD Nacional', 'Funchal']
Gil Vicente --> Gil Vicente --> 100
['Sporting Braga', 'SC Braga', 'Moreirense', 'Moreira de Cónegos', 'Moreirense FC', 'Boavista', 'Boavista Futebol Clube', 'Boavista FC', 'Nacional', 'Clube Desportivo Nacional', 'CD Nacional', 'Funchal', 'Gil Vicente', 'V. Barcelos', 'Gil Vicente FC']
Benfica --> Benfica --> 100
['Sporting Braga', 'SC Braga', 'Moreirense', 'Moreira de Cónegos', 'Moreire

Atlético --> Atletico de Madrid --> 77
-----------------------------
2005
England
['Sporting Braga', 'SC Braga', 'Moreirense', 'Moreira de Cónegos', 'Moreirense FC', 'Boavista', 'Boavista Futebol Clube', 'Boavista FC', 'Nacional', 'Clube Desportivo Nacional', 'CD Nacional', 'Funchal', 'Gil Vicente', 'V. Barcelos', 'Gil Vicente FC', 'Benfica', 'Sport Lisboa Benfica', 'SL Benfica', 'Marítimo', 'Marítimo da Madeira', 'CS Marítimo', 'C. Funchal', 'Rio Ave', 'Rio Ave FC', 'SC Beira-Mar', 'SC Beira Mar', 'União Leiria', 'União Desportivo de Leiria', 'União de Leiria, SAD', 'Belenenses', 'CF Os Belenenses', 'Belém', 'C.F. Os Belenenses', 'Os Belenenses', 'FC Porto', 'F.C. Porto', 'Estoril Praia', 'Estoril', 'GD Estoril-Praia', 'F.C. Penafiel', 'FC Penafiel', 'Vitória Futebol Clube', 'Vitória de Setúbal', 'Vitória Setúbal', 'Vitória SC', 'Vitória de Guimarães', 'Vitória Guimarães', 'Sporting Lisbon', 'Sporting CP Lisbon', 'Sporting CP', 'Coimbra', 'Académica de Coimbra', 'Académica Coimbra', '

Fiorentina --> Firenze --> 59
-----------------------------
2005
Germany
['Sporting Braga', 'SC Braga', 'Moreirense', 'Moreira de Cónegos', 'Moreirense FC', 'Boavista', 'Boavista Futebol Clube', 'Boavista FC', 'Nacional', 'Clube Desportivo Nacional', 'CD Nacional', 'Funchal', 'Gil Vicente', 'V. Barcelos', 'Gil Vicente FC', 'Benfica', 'Sport Lisboa Benfica', 'SL Benfica', 'Marítimo', 'Marítimo da Madeira', 'CS Marítimo', 'C. Funchal', 'Rio Ave', 'Rio Ave FC', 'SC Beira-Mar', 'SC Beira Mar', 'União Leiria', 'União Desportivo de Leiria', 'União de Leiria, SAD', 'Belenenses', 'CF Os Belenenses', 'Belém', 'C.F. Os Belenenses', 'Os Belenenses', 'FC Porto', 'F.C. Porto', 'Estoril Praia', 'Estoril', 'GD Estoril-Praia', 'F.C. Penafiel', 'FC Penafiel', 'Vitória Futebol Clube', 'Vitória de Setúbal', 'Vitória Setúbal', 'Vitória SC', 'Vitória de Guimarães', 'Vitória Guimarães', 'Sporting Lisbon', 'Sporting CP Lisbon', 'Sporting CP', 'Coimbra', 'Académica de Coimbra', 'Académica Coimbra', 'Real Zara

2006
Portugal
['Sporting Braga', 'SC Braga', 'Moreirense', 'Moreira de Cónegos', 'Moreirense FC', 'Boavista', 'Boavista Futebol Clube', 'Boavista FC', 'Nacional', 'Clube Desportivo Nacional', 'CD Nacional', 'Funchal', 'Gil Vicente', 'V. Barcelos', 'Gil Vicente FC', 'Benfica', 'Sport Lisboa Benfica', 'SL Benfica', 'Marítimo', 'Marítimo da Madeira', 'CS Marítimo', 'C. Funchal', 'Rio Ave', 'Rio Ave FC', 'SC Beira-Mar', 'SC Beira Mar', 'União Leiria', 'União Desportivo de Leiria', 'União de Leiria, SAD', 'Belenenses', 'CF Os Belenenses', 'Belém', 'C.F. Os Belenenses', 'Os Belenenses', 'FC Porto', 'F.C. Porto', 'Estoril Praia', 'Estoril', 'GD Estoril-Praia', 'F.C. Penafiel', 'FC Penafiel', 'Vitória Futebol Clube', 'Vitória de Setúbal', 'Vitória Setúbal', 'Vitória SC', 'Vitória de Guimarães', 'Vitória Guimarães', 'Sporting Lisbon', 'Sporting CP Lisbon', 'Sporting CP', 'Coimbra', 'Académica de Coimbra', 'Académica Coimbra', 'Real Zaragoza', 'Real Zaragoza S.A.D.', 'Osasuna', 'Club Atlético Os

['Sporting Braga', 'SC Braga', 'Moreirense', 'Moreira de Cónegos', 'Moreirense FC', 'Boavista', 'Boavista Futebol Clube', 'Boavista FC', 'Nacional', 'Clube Desportivo Nacional', 'CD Nacional', 'Funchal', 'Gil Vicente', 'V. Barcelos', 'Gil Vicente FC', 'Benfica', 'Sport Lisboa Benfica', 'SL Benfica', 'Marítimo', 'Marítimo da Madeira', 'CS Marítimo', 'C. Funchal', 'Rio Ave', 'Rio Ave FC', 'SC Beira-Mar', 'SC Beira Mar', 'União Leiria', 'União Desportivo de Leiria', 'União de Leiria, SAD', 'Belenenses', 'CF Os Belenenses', 'Belém', 'C.F. Os Belenenses', 'Os Belenenses', 'FC Porto', 'F.C. Porto', 'Estoril Praia', 'Estoril', 'GD Estoril-Praia', 'F.C. Penafiel', 'FC Penafiel', 'Vitória Futebol Clube', 'Vitória de Setúbal', 'Vitória Setúbal', 'Vitória SC', 'Vitória de Guimarães', 'Vitória Guimarães', 'Sporting Lisbon', 'Sporting CP Lisbon', 'Sporting CP', 'Coimbra', 'Académica de Coimbra', 'Académica Coimbra', 'Real Zaragoza', 'Real Zaragoza S.A.D.', 'Osasuna', 'Club Atlético Osasuna', 'CA Os

...

['Sporting Braga', 'SC Braga', 'Moreirense', 'Moreira de Cónegos', 'Moreirense FC', 'Boavista', 'Boavista Futebol Clube', 'Boavista FC', 'Nacional', 'Clube Desportivo Nacional', 'CD Nacional', 'Funchal', 'Gil Vicente', 'V. Barcelos', 'Gil Vicente FC', 'Benfica', 'Sport Lisboa Benfica', 'SL Benfica', 'Marítimo', 'Marítimo da Madeira', 'CS Marítimo', 'C. Funchal', 'Rio Ave', 'Rio Ave FC', 'SC Beira-Mar', 'SC Beira Mar', 'União Leiria', 'União Desportivo de Leiria', 'União de Leiria, SAD', 'Belenenses', 'CF Os Belenenses', 'Belém', 'C.F. Os Belenenses', 'Os Belenenses', 'FC Porto', 'F.C. Porto', 'Estoril Praia', 'Estoril', 'GD Estoril-Praia', 'F.C. Penafiel', 'FC Penafiel', 'Vitória Futebol Clube', 'Vitória de Setúbal', 'Vitória Setúbal', 'Vitória SC', 'Vitória de Guimarães', 'Vitória Guimarães', 'Sporting Lisbon', 'Sporting CP Lisbon', 'Sporting CP', 'Coimbra', 'Académica de Coimbra', 'Académica Coimbra', 'Real Zaragoza', 'Real Zaragoza S.A.D.', 'Osasuna', 'Club Atlético Osasuna', 'CA Os

-----------------------------
2019
France
['Sporting Braga', 'SC Braga', 'Moreirense', 'Moreira de Cónegos', 'Moreirense FC', 'Boavista', 'Boavista Futebol Clube', 'Boavista FC', 'Nacional', 'Clube Desportivo Nacional', 'CD Nacional', 'Funchal', 'Gil Vicente', 'V. Barcelos', 'Gil Vicente FC', 'Benfica', 'Sport Lisboa Benfica', 'SL Benfica', 'Marítimo', 'Marítimo da Madeira', 'CS Marítimo', 'C. Funchal', 'Rio Ave', 'Rio Ave FC', 'SC Beira-Mar', 'SC Beira Mar', 'União Leiria', 'União Desportivo de Leiria', 'União de Leiria, SAD', 'Belenenses', 'CF Os Belenenses', 'Belém', 'C.F. Os Belenenses', 'Os Belenenses', 'FC Porto', 'F.C. Porto', 'Estoril Praia', 'Estoril', 'GD Estoril-Praia', 'F.C. Penafiel', 'FC Penafiel', 'Vitória Futebol Clube', 'Vitória de Setúbal', 'Vitória Setúbal', 'Vitória SC', 'Vitória de Guimarães', 'Vitória Guimarães', 'Sporting Lisbon', 'Sporting CP Lisbon', 'Sporting CP', 'Coimbra', 'Académica de Coimbra', 'Académica Coimbra', 'Real Zaragoza', 'Real Zaragoza S.A.D.', 

-----------------------------
2020
Italy
-----------------------------
2020
Germany
['Sporting Braga', 'SC Braga', 'Moreirense', 'Moreira de Cónegos', 'Moreirense FC', 'Boavista', 'Boavista Futebol Clube', 'Boavista FC', 'Nacional', 'Clube Desportivo Nacional', 'CD Nacional', 'Funchal', 'Gil Vicente', 'V. Barcelos', 'Gil Vicente FC', 'Benfica', 'Sport Lisboa Benfica', 'SL Benfica', 'Marítimo', 'Marítimo da Madeira', 'CS Marítimo', 'C. Funchal', 'Rio Ave', 'Rio Ave FC', 'SC Beira-Mar', 'SC Beira Mar', 'União Leiria', 'União Desportivo de Leiria', 'União de Leiria, SAD', 'Belenenses', 'CF Os Belenenses', 'Belém', 'C.F. Os Belenenses', 'Os Belenenses', 'FC Porto', 'F.C. Porto', 'Estoril Praia', 'Estoril', 'GD Estoril-Praia', 'F.C. Penafiel', 'FC Penafiel', 'Vitória Futebol Clube', 'Vitória de Setúbal', 'Vitória Setúbal', 'Vitória SC', 'Vitória de Guimarães', 'Vitória Guimarães', 'Sporting Lisbon', 'Sporting CP Lisbon', 'Sporting CP', 'Coimbra', 'Académica de Coimbra', 'Académica Coimbra',

2021
Spain
-----------------------------
2021
England
['Sporting Braga', 'SC Braga', 'Moreirense', 'Moreira de Cónegos', 'Moreirense FC', 'Boavista', 'Boavista Futebol Clube', 'Boavista FC', 'Nacional', 'Clube Desportivo Nacional', 'CD Nacional', 'Funchal', 'Gil Vicente', 'V. Barcelos', 'Gil Vicente FC', 'Benfica', 'Sport Lisboa Benfica', 'SL Benfica', 'Marítimo', 'Marítimo da Madeira', 'CS Marítimo', 'C. Funchal', 'Rio Ave', 'Rio Ave FC', 'SC Beira-Mar', 'SC Beira Mar', 'União Leiria', 'União Desportivo de Leiria', 'União de Leiria, SAD', 'Belenenses', 'CF Os Belenenses', 'Belém', 'C.F. Os Belenenses', 'Os Belenenses', 'FC Porto', 'F.C. Porto', 'Estoril Praia', 'Estoril', 'GD Estoril-Praia', 'F.C. Penafiel', 'FC Penafiel', 'Vitória Futebol Clube', 'Vitória de Setúbal', 'Vitória Setúbal', 'Vitória SC', 'Vitória de Guimarães', 'Vitória Guimarães', 'Sporting Lisbon', 'Sporting CP Lisbon', 'Sporting CP', 'Coimbra', 'Académica de Coimbra', 'Académica Coimbra', 'Real Zaragoza', 'Real Zarago

-----------------------------
CPU times: user 5.77 s, sys: 145 ms, total: 5.92 s
Wall time: 5.84 s


In [255]:
d['Köln']

[['Hertha BSC Berlin', 'Hertha BSC', 'Hertha Berlin']]

In [239]:
d

defaultdict(list,
            {'Sporting Braga': [['Sporting Braga', 'SC Braga'], 2005],
             'Moreirense': [['Moreirense',
               'Moreira de Cónegos',
               'Moreirense FC'],
              2005],
             'Boavista': [['Boavista',
               'Boavista Futebol Clube',
               'Boavista FC'],
              2005],
             'Nacional': [['Nacional',
               'Clube Desportivo Nacional',
               'CD Nacional',
               'Funchal'],
              2005],
             'Gil Vicente': [['Gil Vicente', 'V. Barcelos', 'Gil Vicente FC'],
              2005],
             'Benfica': [['Benfica', 'Sport Lisboa Benfica', 'SL Benfica'],
              2005],
             'Marítimo': [['Marítimo',
               'Marítimo da Madeira',
               'CS Marítimo',
               'C. Funchal'],
              2005],
             'Rio Ave': [['Rio Ave', 'Rio Ave FC'], 2005],
             'Beira Mar SC': [['SC Beira-Mar', 'SC Beira Mar'], 2005],

In [260]:
j = [['Sporting Braga', 'SC Braga'], ['Moreirense', 'Moreira de Cónegos', 'Moreirense FC'], ['Boavista', 'Boavista Futebol Clube', 'Boavista FC'], ['Nacional', 'Clube Desportivo Nacional', 'CD Nacional', 'Funchal'], ['Gil Vicente', 'V. Barcelos', 'Gil Vicente FC'], ['Benfica', 'Sport Lisboa Benfica', 'SL Benfica'], ['Marítimo', 'Marítimo da Madeira', 'CS Marítimo', 'C. Funchal'], ['Rio Ave', 'Rio Ave FC'], ['SC Beira-Mar', 'SC Beira Mar'], ['União Leiria', 'União Desportivo de Leiria', 'União de Leiria, SAD'], ['Belenenses', 'CF Os Belenenses', 'Belém', 'C.F. Os Belenenses', 'Os Belenenses'], ['FC Porto', 'F.C. Porto'], ['Estoril Praia', 'Estoril', 'GD Estoril-Praia'], ['F.C. Penafiel', 'FC Penafiel'], ['Vitória Futebol Clube', 'Vitória de Setúbal', 'Vitória Setúbal'], ['Vitória SC', 'Vitória de Guimarães', 'Vitória Guimarães'], ['Sporting Lisbon', 'Sporting CP Lisbon', 'Sporting CP'], ['Coimbra', 'Académica de Coimbra', 'Académica Coimbra'], ['Real Zaragoza', 'Real Zaragoza S.A.D.'], ['Osasuna', 'Club Atlético Osasuna', 'CA Osasuna'], ['Levante', 'Levante U.D.', 'Levante Unión Deportiva S.A.D.', 'Levante Unión Deportiva', 'Levante UD'], ['Real Betis', 'Real Betis Balompié S.A.D.', 'Real Betis Balompié'], ['Real Madrid', 'Real Madrid C.F.', 'Real Madrid Club de Fútbol', 'Real Madrid CF'], ['Numancia', 'C.D. Numancia'], ['Villarreal', 'Villarreal C.F.', 'Villarreal Club de Fútbol S.A.D.', 'Villarreal Club de Fútbol', 'Villarreal CF'], ['Valencia', 'Valencia C.F.', 'Valencia Club de Fútbol S.A.D.', 'Valencia Club de Fútbol', 'Valencia CF'], ['Albacete'], ['Getafe', 'Getafe C.F.', 'Getafe Club de Fútbol S.A.D.', 'Getafe Club de Fútbol', 'Getafe CF'], ['Sevilla', 'Sevilla F.C.', 'Sevilla Fútbol Club S.A.D.', 'Sevilla Fútbol Club', 'Sevilla FC'], ['FC Barcelona', 'F.C. Barcelona'], ['RCD Mallorca', 'R.C.D. Mallorca', 'Real Club Deportivo Mallorca S.A.D.', 'Real Club Deportivo Mallorca'], ['RC Deportivo La Coruna', 'R.C. Deportivo La Coruna', 'R.C. Deportivo', 'Real Club Deportivo de La Coruña S.A.D.', 'Real Club Deportivo de La Coruña', 'RC Deportivo de La Coruña'], ['RCD Espanyol', 'R.C.D. Espanyol', 'R.C.D. Espanyol de Barcelona S.A.D.', 'RCD Espanyol de Barcelona'], ['Athletic de Bilbao', 'Athletic Club', 'Athletic Club de Bilbao'], ['Málaga CF', 'Málaga C.F.', 'Málaga Club de Fútbol S.A.D.', 'Málaga Club de Fútbol'], ['Racing de Santander', 'Real Racing Club', 'Real Racing Club S.A.D.'], ['Real Sociedad', 'Real Sociedad de Fútbol S.A.D.', 'Real Sociedad de Fútbol'], ['Atletico de Madrid', 'Atlético Madrid', 'Atlético de Madrid', 'Club Atlético de Madrid S.A.D.', 'Club Atlético de Madrid'], ['Portsmouth'], ['Bolton Wanderers'], ['Aston Villa'], ['Arsenal', 'Arsenal FC'], ['Southampton'], ['Crystal Palace'], ['Charlton Athletic'], ['Middlesbrough'], ['Blackburn Rovers'], ['Everton'], ['Tottenham Hotspur'], ['Fulham'], ['Norwich City'], ['Liverpool'], ['Birmingham City'], ['Chelsea', 'Chelsea FC'], ['West Bromwich', 'West Bromwich Albion'], ['Newcastle United'], ['Manchester City'], ['Manchester United'], ['Brescia'], ['Messina'], ['Juventus'], ['Lecce'], ['Livorno'], ['Cagliari'], ['Bologna'], ['Parma'], ['Lazio'], ['Siena'], ['Udinese'], ['Atalanta BC', 'Atalanta'], ['Palermo'], ['AS Roma', 'Roma'], ['Inter Milan', 'Inter'], ['Chievo Verona'], ['AC Milan', 'Milan'], ['Doria', 'Sampdoria'], ['Reggina Calcio', 'Reggina'], ['Firenze', 'Fiorentina', 'ACF Fiorentina'], ['Arminia Bielefeld', 'DSC Arminia Bielefeld'], ['SC Freiburg', 'Sport-Club Freiburg'], ['Hannover 96'], ['VfL Bochum'], ['FC Kaiserslautern', '1. FC Kaiserslautern'], ['FC Nürnberg', '1. FC Nürnberg', '1. FC Nuremberg'], ['VfL Wolfsburg'], ['FC Schalke 04'], ['VfB Stuttgart'], ['FC Hansa Rostock'], ['SV Werder Bremen', 'Werder Bremen'], ['Mainz', '1. FSV Mainz 05'], ['Hertha BSC Berlin', 'Hertha BSC', 'Hertha Berlin'], ['Monchengladbach', "Borussia M'gladbach", 'Borussia Mönchengladbach'], ['Borussia Dortmund'], ['Hamburger Sport Verein', 'Hamburg SV', 'Hamburger SV'], ['Bayer Leverkusen', 'Bayer 04 Leverkusen'], ['Bayern Munich', 'Bayern München', 'FC Bayern Munich', 'FC Bayern München'], ['Istres'], ['Olympique Lyonnais'], ['Sochaux', 'FC Sochaux', 'FC Sochaux-Montbéliard'], ['Toulouse', 'Toulouse F.C.', 'Toulouse FC', 'Toulouse Football Club'], ['AC Ajaccio'], ['AJ Auxerre'], ['Bordeaux', 'FC Girondins Bordeaux', 'FCGB', 'Girondins de Bordeaux', 'FC Girondins de Bordeaux'], ['SC Bastia', 'Sporting Club Bastia'], ['SM Caen', 'Stade Malherbe Caen'], ['Marseille', 'Olympique de Marseille'], ['OGC Nice', "OGC Nice Côte D'azur"], ['Lille Osc', 'Lille OSC', 'LOSC Lille', 'LOSC Lille Métropole'], ['FC Metz', 'Football Club de Metz'], ['RC Lens', 'Racing Club de Lens'], ['FC Nantes'], ['AS Monaco', 'AS Monaco FC', 'AS Monaco Football Club SA'], ['AS Saint-Etienne', 'A.S. Saint-Etienne', 'AS Saint-Étienne'], ['RC Strasburg', 'RC Strasbourg', 'RC Strasbourg Alsace'], ['Rennes', 'Stade Rennais', 'Stade Rennais FC'], ['Paris Saint-Germain'], ['Estrela Amadora'], ['Naval', 'Naval 1° de Maio'], ['Paços de Ferreira', 'Paços Ferreira', 'FC Paços de Ferreira'], ['Cádiz', 'Cádiz CF'], ['Deport. Alavés', 'Deportivo Alavés', 'D. Alavés'], ['RC Celta Vigo', 'Real Club Celta de Vigo', 'RC Celta de Vigo', 'RC Celta'], ['Wigan Athletic'], ['Sunderland'], ['West Ham United'], ['Ascoli'], ['Empoli'], ['Treviso'], ['Eintracht Frankfurt'], ['MSV Duisburg']]
i = sum(j,[])
i

['Sporting Braga',
 'SC Braga',
 'Moreirense',
 'Moreira de Cónegos',
 'Moreirense FC',
 'Boavista',
 'Boavista Futebol Clube',
 'Boavista FC',
 'Nacional',
 'Clube Desportivo Nacional',
 'CD Nacional',
 'Funchal',
 'Gil Vicente',
 'V. Barcelos',
 'Gil Vicente FC',
 'Benfica',
 'Sport Lisboa Benfica',
 'SL Benfica',
 'Marítimo',
 'Marítimo da Madeira',
 'CS Marítimo',
 'C. Funchal',
 'Rio Ave',
 'Rio Ave FC',
 'SC Beira-Mar',
 'SC Beira Mar',
 'União Leiria',
 'União Desportivo de Leiria',
 'União de Leiria, SAD',
 'Belenenses',
 'CF Os Belenenses',
 'Belém',
 'C.F. Os Belenenses',
 'Os Belenenses',
 'FC Porto',
 'F.C. Porto',
 'Estoril Praia',
 'Estoril',
 'GD Estoril-Praia',
 'F.C. Penafiel',
 'FC Penafiel',
 'Vitória Futebol Clube',
 'Vitória de Setúbal',
 'Vitória Setúbal',
 'Vitória SC',
 'Vitória de Guimarães',
 'Vitória Guimarães',
 'Sporting Lisbon',
 'Sporting CP Lisbon',
 'Sporting CP',
 'Coimbra',
 'Académica de Coimbra',
 'Académica Coimbra',
 'Real Zaragoza',
 'Real Zarago

In [262]:
team = 'Hertha BSC Berlin'
team in i

True

In [157]:
lista1=['Marítimo', 'Feirense', 'Gil Vicente']

lista2=['F. Santa Maria da Feira', 'V. Barcelos', 'C. Funchal']

for team in lista1:
    print(team,process.extract(team, lista2,limit=10))
    print('\n')

Marítimo [('F. Santa Maria da Feira', 51), ('V. Barcelos', 39), ('C. Funchal', 12)]


Feirense [('F. Santa Maria da Feira', 45), ('V. Barcelos', 32), ('C. Funchal', 23)]


Gil Vicente [('F. Santa Maria da Feira', 31), ('V. Barcelos', 28), ('C. Funchal', 19)]




In [152]:
lista2=['Marítimo', 'Feirense', 'Gil Vicente']

lista1=['F. Santa Maria da Feira', 'V. Barcelos', 'C. Funchal']

for team in lista1:
    print(process.extract(team, lista2,limit=10))
    print('\n')

[('Marítimo', 51), ('Feirense', 45), ('Gil Vicente', 31)]


[('Marítimo', 39), ('Feirense', 32), ('Gil Vicente', 28)]


[('Feirense', 23), ('Gil Vicente', 19), ('Marítimo', 12)]




In [62]:
for year in Years:
    for country in Countries:
        original_teams_list_year_country = list(set(
            list(df1[(df1['Year'] == year) & 
                     (df1['Country'] == country)]
                     ['Home_team']) +\
            list(df1[(df1['Year'] == year) & 
                     (df1['Country'] == country)]
                     ['Away_team'])))

        fifa_teams_list_year_country = list(set(
            list(table_fifa_all_years[(table_fifa_all_years['Year'] == year) & 
                     (table_fifa_all_years['Country'] == country)]
                     ['Name'])))
        
        print(year)
        print(country)
        if len(original_teams_list_year_country) == len(fifa_teams_list_year_country):
            print('OK')
            print('\n')
        else:
            print('********************************************************')
            print('********************************************************')
            print('********************************************************')
            print(f'ERROR: {year},{country},\n\
            Number of teams original table: {len(original_teams_list_year_country)},\n\
            Number of teams fifa table: {len(fifa_teams_list_year_country)}')
            print('********************************************************')
            print('********************************************************')
            print('********************************************************')
            print('\n')

2005
Portugal
OK


2005
Spain
OK


2005
England
OK


2005
Italy
OK


2005
Germany
OK


2005
France
OK


2006
Portugal
OK


2006
Spain
OK


2006
England
OK


2006
Italy
OK


2006
Germany
OK


2006
France
OK


2007
Portugal
OK


2007
Spain
OK


2007
England
OK


2007
Italy
OK


2007
Germany
OK


2007
France
OK


2008
Portugal
OK


2008
Spain
OK


2008
England
OK


2008
Italy
OK


2008
Germany
OK


2008
France
OK


2009
Portugal
OK


2009
Spain
OK


2009
England
OK


2009
Italy
OK


2009
Germany
OK


2009
France
OK


2010
Portugal
OK


2010
Spain
OK


2010
England
OK


2010
Italy
OK


2010
Germany
OK


2010
France
OK


2011
Portugal
OK


2011
Spain
OK


2011
England
OK


2011
Italy
OK


2011
Germany
OK


2011
France
OK


2012
Portugal
OK


2012
Spain
OK


2012
England
OK


2012
Italy
OK


2012
Germany
OK


2012
France
OK


2013
Portugal
OK


2013
Spain
OK


2013
England
OK


2013
Italy
OK


2013
Germany
OK


2013
France
OK


2014
Portugal
OK


2014
Spain
OK


2014
England
OK


2014
Italy


In [63]:
lista1=['PSG', 'Strasbourg', 'Saint-Étienne', 'Stade Rennais']

lista2=['AS Saint-Etienne', 'RC Strasburg', 'Paris Saint-Germain', 
        'Le Havre AC', 'Rennes', 'AS Nancy-Lorraine', 
        'En Avant Guingamp', 'ES Troyes AC']

for team in lista1:
    print(process.extract(team, lista2,limit=10))
    print('\n')

[('AS Saint-Etienne', 30), ('RC Strasburg', 30), ('Paris Saint-Germain', 30), ('Rennes', 30), ('AS Nancy-Lorraine', 30), ('En Avant Guingamp', 30), ('ES Troyes AC', 30), ('Le Havre AC', 0)]


[('RC Strasburg', 82), ('ES Troyes AC', 36), ('Rennes', 30), ('AS Nancy-Lorraine', 30), ('Paris Saint-Germain', 28), ('En Avant Guingamp', 27), ('Le Havre AC', 19), ('AS Saint-Etienne', 18)]


[('AS Saint-Etienne', 86), ('Paris Saint-Germain', 86), ('Rennes', 66), ('AS Nancy-Lorraine', 48), ('En Avant Guingamp', 46), ('Le Havre AC', 33), ('ES Troyes AC', 33), ('RC Strasburg', 17)]


[('Rennes', 60), ('Paris Saint-Germain', 48), ('Le Havre AC', 42), ('AS Saint-Etienne', 41), ('RC Strasburg', 40), ('AS Nancy-Lorraine', 40), ('ES Troyes AC', 40), ('En Avant Guingamp', 33)]




In [36]:
process.extract?

In [9]:
d

defaultdict(list,
            {'Os Belenenses': ['Belenenses',
              'CF Os Belenenses',
              'Belém',
              'C.F. Os Belenenses',
              'Os Belenenses'],
             'Rio Ave': ['Rio Ave', 'Rio Ave FC'],
             'Estoril': ['Estoril Praia', 'Estoril', 'GD Estoril-Praia'],
             'Boavista': ['Boavista', 'Boavista Futebol Clube', 'Boavista FC'],
             'Nacional': ['Nacional',
              'Clube Desportivo Nacional',
              'CD Nacional',
              'Funchal'],
             'Gil Vicente': ['Gil Vicente',
              'F. Santa Maria da Feira',
              'Moreira de Cónegos',
              'SC Braga',
              'Gil Vicente FC'],
             'Porto': ['FC Porto', 'F.C. Porto'],
             'Benfica': ['Benfica', 'Sport Lisboa Benfica', 'SL Benfica'],
             'Penafiel': ['F.C. Penafiel', 'FC Penafiel'],
             'Moreirense': ['Moreirense',
              'Moreira de Cónegos',
              'Moreirense FC'

In [119]:
lista_a = ['A','B','C']

In [120]:
lista_a.remove('B')
lista_a

['A', 'C']

In [122]:
1+1

2

In [ ]:
import collections
b = collections.defaultdict(list)
b

In [41]:
import collections
a = collections.defaultdict(list)
a['Porto'].append('Porto')
a['Porto'].append('Porto')
a['Porto'].append('FC Porto')
a['Porto']
a['Benfica'].append('SL Benfica')

In [42]:
a

defaultdict(list,
            {'Porto': ['Porto', 'Porto', 'FC Porto'],
             'Benfica': ['SL Benfica']})

In [45]:
a.keys()

dict_keys(['Porto', 'Benfica'])

In [46]:
a.values()

dict_values([['Porto', 'Porto', 'FC Porto'], ['SL Benfica']])

In [53]:
team = 'Porto'
team in a.values()

False

In [54]:
h = sum(a.values(),[])
h

['Porto', 'Porto', 'FC Porto', 'SL Benfica']

In [57]:
team = 'Porto'
team in sum(a.values(),[])

True

In [43]:
m = 'SL Benfica'
keys = [k for k, v in a.items() if m in v]
print(keys)

['Benfica']


In [58]:
j= ['Porto', 'Porto', 'FC Porto'] + ['SL Benfica']
j

['Porto', 'Porto', 'FC Porto', 'SL Benfica']

In [ ]:
a.items()

In [ ]:
b = collections.defaultdict(list)
b['Real Madrid'].append('Real Madrid')
b.items()

In [ ]:
import collections
a = collections.defaultdict(list)
a['Porto'].append('Porto')
a['Porto'].append('FC Porto')
a['Porto']
a['Benfica'].append('SL Benfica')

In [ ]:
team = 'Porto'
if team not in a['Porto']:
    a['Porto'].append(team)

a

In [ ]:
team = 'F.C. Porto'
if team not in a['Porto']:
    a['Porto'].append(team)

a

In [ ]:
team = 'Porto'
if team not in a['Porto']:
    a['Porto'].append(team)

a

In [ ]:
round(2.5)

In [ ]:
round(1.5)

In [ ]:
round(3.5)

In [ ]:
round(4.5)

In [ ]:
round(2.5+0.00001)